In [ ]:
import pandas as pd
final_data = pd.read_csv("./words_data.csv", encoding='utf-8')
final_data = final_data = final_data.loc[:,['header', 'detail']]
final_data

test_data = pd.read_csv("./programmers_data.csv", encoding='utf-8')
print(len(test_data))

3262


In [ ]:
# 본 코드
import os
import time
import datetime
from tensorflow import flags
import tensorflow as tf
import numpy as np
import cnn_tool as tool

In [ ]:
import sys
print(sys.version)

3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]


In [ ]:
# !pip install tensorflow==1.15

In [ ]:
# !pip install flags

In [ ]:
class TextCNN(object):
    """
    A CNN for text classification.
    Uses an embedding layer, followed by a convolutional, max-pooling and softmax layer.
    <Parameters>
        - sequence_length: 최대 문장 길이
        - num_classes: 클래스 개수
        - vocab_size: 등장 단어 수
        - embedding_size: 각 단어에 해당되는 임베디드 벡터의 차원
        - filter_sizes: convolutional filter들의 사이즈 (= 각 filter가 몇 개의 단어를 볼 것인가?) (예: "3, 4, 5")
        - num_filters: 각 filter size 별 filter 수
        - l2_reg_lambda: 각 weights, biases에 대한 l2 regularization 정도
    """

    def __init__(
            self, sequence_length, num_classes, vocab_size,
            embedding_size, filter_sizes, num_filters, l2_reg_lambda=0.0):
        # Placeholders for input, output and dropout
        self.input_x = tf.placeholder(tf.int32, [None, sequence_length], name="input_x")
        self.input_y = tf.placeholder(tf.float32, [None, num_classes], name="input_y")
        self.dropout_keep_prob = tf.placeholder(tf.float32, name="dropout_keep_prob")

        # Keeping track of l2 regularization loss (optional)
        l2_loss = tf.constant(0.0)

        # Embedding layer
        """
        <Variable>
            - W: 각 단어의 임베디드 벡터의 성분을 랜덤하게 할당
        """
        with tf.device('/cpu:0'), tf.name_scope("embedding"):
        #with tf.device('/cpu:0'), tf.name_scope("embedding"):
            W = tf.Variable(
                tf.random_uniform([vocab_size, embedding_size], -1.0, 1.0),
                name="W")
            self.embedded_chars = tf.nn.embedding_lookup(W, self.input_x)
            self.embedded_chars_expanded = tf.expand_dims(self.embedded_chars, -1)

        # Create a convolution + maxpool layer for each filter size
        pooled_outputs = []
        for i, filter_size in enumerate(filter_sizes):
            with tf.name_scope("conv-maxpool-%s" % filter_size):
                # Convolution Layer
                filter_shape = [filter_size, embedding_size, 1, num_filters]
                W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name="W")
                b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name="b")
                conv = tf.nn.conv2d(
                    self.embedded_chars_expanded,
                    W,
                    strides=[1, 1, 1, 1],
                    padding="VALID",
                    name="conv")
                # Apply nonlinearity
                h = tf.nn.relu(tf.nn.bias_add(conv, b), name="relu")
                # Maxpooling over the outputs
                pooled = tf.nn.max_pool(
                    h,
                    ksize=[1, sequence_length - filter_size + 1, 1, 1],
                    strides=[1, 1, 1, 1],
                    padding='VALID',
                    name="pool")
                pooled_outputs.append(pooled)

        # Combine all the pooled features
        num_filters_total = num_filters * len(filter_sizes)
        self.h_pool = tf.concat(pooled_outputs,3)
        self.h_pool_flat = tf.reshape(self.h_pool, [-1, num_filters_total])

        # Add dropout
        with tf.name_scope("dropout"):
            self.h_drop = tf.nn.dropout(self.h_pool_flat, self.dropout_keep_prob)

        # Final (unnormalized) scores and predictions
        with tf.name_scope("output"):
            W = tf.get_variable(
                "W",
                shape=[num_filters_total, num_classes],
                initializer=tf.contrib.layers.xavier_initializer())
            b = tf.Variable(tf.constant(0.1, shape=[num_classes]), name="b")
            l2_loss += tf.nn.l2_loss(W)
            l2_loss += tf.nn.l2_loss(b)
            self.scores = tf.nn.xw_plus_b(self.h_drop, W, b, name="scores")
            self.predictions = tf.argmax(self.scores, 1, name="predictions")

        # Calculate Mean cross-entropy loss
        with tf.name_scope("loss"):
            losses = tf.nn.softmax_cross_entropy_with_logits(logits=self.scores, labels=self.input_y)
            self.loss = tf.reduce_mean(losses) + l2_reg_lambda * l2_loss

        # Accuracy
        with tf.name_scope("accuracy"):
            correct_predictions = tf.equal(self.predictions, tf.argmax(self.input_y, 1))
            self.accuracy = tf.reduce_mean(tf.cast(correct_predictions, "float"), name="accuracy")

In [ ]:
# data loading
import pandas as pd

df = pd.read_csv("./words_data.csv", encoding='utf-8')
df = df.loc[:,['header', 'detail']]


In [ ]:
contents = df['detail']
points = df['header']
contents

0        ['webgl', 'canvas', '활용', 'd', 'd', '모델', '인터'...
1        ['ruby', 'on', 'rails', '이용', 'rest', 'api', '...
2        ['한국', '미국', '하루', '평균', '사용', '블라인드', '서비스', ...
3        ['fts', 'fulfillment', 'and', 'transportation'...
4        ['agile', '골', '라라', '개발', '문화', '백엔드', '개발', ...
                               ...                        
46216    ['자체', '개발', 'web', 'logging', 'framework', '이...
46217    ['ai', '를', '기반', '개인', '취향', '최적화', '음악', '추천...
46218    ['data', 'engineer', 'python', '클라우드', '환경', '...
46219    ['senior', 'frontend', 'engineer', 'mlops', '제...
46220    ['국내', '유명', 'ai', '기반', '오디오', '기술', '스타트업', ...
Name: detail, Length: 46221, dtype: object

In [ ]:
def make_output(points):
  results = np.zeros((len(points),10))
  for idx, point in enumerate(points):
    results[idx, point] = 1
  return results

In [ ]:
# tranform document to vector
max_document_length = 200
x, vocabulary, vocab_size = tool.make_input(contents,max_document_length)
print('사전단어수 : %s' % (vocab_size))
y = make_output(points)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
Instructions for updating:
Please use tensorflow/transform or tf.data.
사전단어수 : 36112


In [ ]:
# divide dataset into train/test set
x_train, x_test, y_train, y_test = tool.divide(x,y,train_prop=0.8)

In [ ]:
# Model Hyperparameters
flags.DEFINE_integer("embedding_dim", 128, "Dimensionality of embedded vector (default: 128)")
flags.DEFINE_string("filter_sizes", "3,4,5", "Comma-separated filter sizes (default: '3,4,5')")
flags.DEFINE_integer("num_filters", 128, "Number of filters per filter size (default: 128)")
flags.DEFINE_float("dropout_keep_prob", 0.5, "Dropout keep probability (default: 0.5)")
flags.DEFINE_float("l2_reg_lambda", 0.1, "L2 regularization lambda (default: 0.0)")

In [ ]:
# Training parameters
flags.DEFINE_integer("batch_size", 64, "Batch Size (default: 64)")
flags.DEFINE_integer("num_epochs", 10, "Number of training epochs (default: 200)")
flags.DEFINE_integer("evaluate_every", 100, "Evaluate model on dev set after this many steps (default: 100)")
flags.DEFINE_integer("checkpoint_every", 100, "Save model after this many steps (default: 100)")
flags.DEFINE_integer("num_checkpoints", 5, "Number of checkpoints to store (default: 5)")

In [ ]:
# Misc Parameters
flags.DEFINE_boolean("allow_soft_placement", True, "Allow device soft device placement")
flags.DEFINE_boolean("log_device_placement", False, "Log placement of ops on devices")

In [ ]:
flags.DEFINE_string("f", "", "kernel")

FLAGS = tf.flags.FLAGS
#FLAGS._parse_flags()

import sys
FLAGS(sys.argv)

print("\nParameters:")
for attr, value in sorted(FLAGS.__flags.items()):
    print("{}={}".format(attr.upper(), value))
print("")


Parameters:
ALLOW_SOFT_PLACEMENT=<absl.flags._flag.BooleanFlag object at 0x7f39cf4eda10>
ALSOLOGTOSTDERR=<absl.flags._flag.BooleanFlag object at 0x7f39e9ef2090>
BATCH_SIZE=<absl.flags._flag.Flag object at 0x7f39cf514590>
CHECKPOINT_EVERY=<absl.flags._flag.Flag object at 0x7f39cf514510>
DROPOUT_KEEP_PROB=<absl.flags._flag.Flag object at 0x7f39cf514610>
EMBEDDING_DIM=<absl.flags._flag.Flag object at 0x7f39cf514a50>
EVALUATE_EVERY=<absl.flags._flag.Flag object at 0x7f39cf514b90>
F=<absl.flags._flag.Flag object at 0x7f39cf4f0e10>
FILTER_SIZES=<absl.flags._flag.Flag object at 0x7f39cf514c90>
L2_REG_LAMBDA=<absl.flags._flag.Flag object at 0x7f39cf514850>
LOG_DEVICE_PLACEMENT=<absl.flags._flag.BooleanFlag object at 0x7f39cf4ed990>
LOG_DIR=<absl.flags._flag.Flag object at 0x7f39e9ef2150>
LOGGER_LEVELS=<absl.logging._LoggerLevelsFlag object at 0x7f39e9ef22d0>
LOGTOSTDERR=<absl.flags._flag.BooleanFlag object at 0x7f39e9f6b9d0>
NUM_CHECKPOINTS=<absl.flags._flag.Flag object at 0x7f39cf514190>
NUM

In [ ]:
print(x_train.shape)
print(y_train.shape)
print(y_train.shape[1])

(36977, 200)
(36977, 10)
10


In [ ]:
# 3. train the model and test
with tf.Graph().as_default():
    sess = tf.Session()
    with sess.as_default():
        cnn = TextCNN(sequence_length=x_train.shape[1],
                      num_classes=y_train.shape[1],
                      vocab_size=vocab_size,
                      embedding_size=FLAGS.embedding_dim,
                      filter_sizes=list(map(int, FLAGS.filter_sizes.split(","))),
                      num_filters=FLAGS.num_filters,
                      l2_reg_lambda=FLAGS.l2_reg_lambda)

        # Define Training procedure
        global_step = tf.Variable(0, name="global_step", trainable=False)
        optimizer = tf.train.AdamOptimizer(1e-3)
        grads_and_vars = optimizer.compute_gradients(cnn.loss)
        train_op = optimizer.apply_gradients(grads_and_vars, global_step=global_step)

        # Keep track of gradient values and sparsity (optional)
        grad_summaries = []
        for g, v in grads_and_vars:
            if g is not None:
                grad_hist_summary = tf.summary.histogram("{}".format(v.name), g)
                sparsity_summary = tf.summary.scalar("{}".format(v.name), tf.nn.zero_fraction(g))
                grad_summaries.append(grad_hist_summary)
                grad_summaries.append(sparsity_summary)
        grad_summaries_merged = tf.summary.merge(grad_summaries)

        # Output directory for models and summaries
        timestamp = str(int(time.time()))
        out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
        print("Writing to {}\n".format(out_dir))

        # Summaries for loss and accuracy
        loss_summary = tf.summary.scalar("loss", cnn.loss)
        acc_summary = tf.summary.scalar("accuracy", cnn.accuracy)

        # Train Summaries
        train_summary_op = tf.summary.merge([loss_summary, acc_summary, grad_summaries_merged])
        train_summary_dir = os.path.join(out_dir, "summaries", "train")
        train_summary_writer = tf.summary.FileWriter(train_summary_dir, sess.graph)

        # Dev summaries
        dev_summary_op = tf.summary.merge([loss_summary, acc_summary])
        dev_summary_dir = os.path.join(out_dir, "summaries", "dev")
        dev_summary_writer = tf.summary.FileWriter(dev_summary_dir, sess.graph)

        # Checkpoint directory. Tensorflow assumes this directory already exists so we need to create it
        checkpoint_dir = os.path.abspath(os.path.join(out_dir, "checkpoints"))
        checkpoint_prefix = os.path.join(checkpoint_dir, "model")
        if not os.path.exists(checkpoint_dir):
            os.makedirs(checkpoint_dir)
        saver = tf.train.Saver(tf.global_variables(), max_to_keep=FLAGS.num_checkpoints)

        # Initialize all variables
        sess.run(tf.global_variables_initializer())

        def train_step(x_batch, y_batch):
            """
            A single training step
            """
            feed_dict = {
                cnn.input_x: x_batch,
                cnn.input_y: y_batch,
                cnn.dropout_keep_prob: FLAGS.dropout_keep_prob
            }
            _, step, summaries, loss, accuracy = sess.run(
                [train_op, global_step, train_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            train_summary_writer.add_summary(summaries, step)


        def dev_step(x_batch, y_batch, writer=None):
            """
            Evaluates model on a dev set
            """
            feed_dict = {
                cnn.input_x: x_batch,
                cnn.input_y: y_batch,
                cnn.dropout_keep_prob: 1.0
            }
            step, summaries, loss, accuracy = sess.run(
                [global_step, dev_summary_op, cnn.loss, cnn.accuracy],
                feed_dict)
            time_str = datetime.datetime.now().isoformat()
            print("{}: step {}, loss {:g}, acc {:g}".format(time_str, step, loss, accuracy))
            if writer:
                writer.add_summary(summaries, step)


        def batch_iter(data, batch_size, num_epochs, shuffle=True):
            """
            Generates a batch iterator for a dataset.
            """
            data = np.array(data)
            data_size = len(data)
            num_batches_per_epoch = int((len(data) - 1) / batch_size) + 1
            for epoch in range(num_epochs):
                # Shuffle the data at each epoch
                if shuffle:
                    shuffle_indices = np.random.permutation(np.arange(data_size))
                    shuffled_data = data[shuffle_indices]
                else:
                    shuffled_data = data
                for batch_num in range(num_batches_per_epoch):
                    start_index = batch_num * batch_size
                    end_index = min((batch_num + 1) * batch_size, data_size)
                    yield shuffled_data[start_index:end_index]


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

INFO:tensorflow:Summary name embedding/W:0 is illegal; using embedding/W_0 instead.
INFO:tensorflow:Summary name embedding/W:0 is illegal; using embedding/W_0 instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0 is illegal; using conv-maxpool-3/W_0 instead.
INFO:tensorflow:Summary name conv-maxpool-3/W:0 is illegal; using conv-maxpool-3/W_0 instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0 is illegal; using conv-maxpool-3/b_0 instead.
INFO:tensorflow:Summary name conv-maxpool-3/b:0 is illegal; using conv-maxpool-3/b_0 instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0 is illegal; using conv-maxpool-4/W_0 instead.
INFO:tensorflow:Summary name conv-maxpool-4/W:0 is illeg

In [ ]:
        # Generate batches
        batches = batch_iter(
            list(zip(x_train, y_train)), FLAGS.batch_size, FLAGS.num_epochs)

        testpoint = 0
        # Training loop. For each batch...
        for batch in batches:
            x_batch, y_batch = zip(*batch)
            train_step(x_batch, y_batch)
            current_step = tf.train.global_step(sess, global_step)
            if current_step % FLAGS.evaluate_every == 0:
                if testpoint + 100 < len(x_test):
                    testpoint += 100
                else:
                    testpoint = 0
                print("\nEvaluation:")
                dev_step(x_test[testpoint:testpoint+100], y_test[testpoint:testpoint+100], writer=dev_summary_writer)
                print("")
            if current_step % FLAGS.checkpoint_every == 0:
                path = saver.save(sess, checkpoint_prefix, global_step=current_step)
                print("Saved model checkpoint to {}\n".format(path))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:97: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
2022-05-05T21:12:50.298964: step 6843, loss 1.55651, acc 0.4375
2022-05-05T21:12:50.936559: step 6844, loss 1.62934, acc 0.453125
2022-05-05T21:12:51.563908: step 6845, loss 1.47682, acc 0.46875
2022-05-05T21:12:52.241867: step 6846, loss 1.6152, acc 0.40625
2022-05-05T21:12:52.960129: step 6847, loss 1.42992, acc 0.46875
2022-05-05T21:12:53.667031: step 6848, loss 1.3361, acc 0.515625
2022-05-05T21:12:54.369841: step 6849, loss 1.28052, acc 0.5625
2022-05-05T21:12:55.040329: step 6850, loss 1.35787, acc 0.5
2022-05-05T21:12:55.705224: step 6851, loss 1.55394, acc 0.421875
2022-05-05T21:12:56.342097: step 6852, loss 1.41548, acc 0.546875
2022-05-05T21:12:56.968606: step 6853, loss 1.41225, acc 0.421875
2022-05-05T21:12:57.592130: step 6854, loss 1.35006, acc 0.578125
2022-05-05T21:12:58.216177: step 6855, loss 1.34686, acc 0.5
2022-05-05T21:12:58.824930: step 6856, loss 1.62614, acc 0.484375
2022-05-05T21:12:59.474362: step 6857, loss 1.33005, acc 0.